# Constants:

In [1]:
SEQ_LEN = 60   # how many past candles to use to predict
CANDLES_SHIFT = 5 # how many candles to shift between sequences
NAME = "shift5_decayNorm_seq60_ov40th004"
VALIDATION_PCT = 0.2

# Functions:

compose sequences df

In [2]:
from tqdm import tqdm
import numpy as np
import pandas as pd

# composes dfs of individual sequences from different timescales (in reverse)
# dfs should be ordered fine to coarse with the finest one having the labels
# seqLen should be divisible by the number of time-resolutions len(dfs)
def composeTimeDecaySequences(dfs, timescales):
    numTimeScales = len(dfs)
    seqPartLen = int(SEQ_LEN/numTimeScales)

    sequences = []
    finestEndIndex = len(dfs[0])-1
    for _ in tqdm(range(int((len(dfs[0]) -int(seqPartLen*(sum(timescales))/300))/CANDLES_SHIFT))): # calculating how many sequences there will be
        
        lastTimestamp = dfs[0]["timestamp"].iloc[finestEndIndex]+1
        seqDf = pd.DataFrame()
        for i in range(numTimeScales):

            # get prev index. Last with timestamp < lastTimestamp
            endIndex = dfs[i][dfs[i]["timestamp"] < lastTimestamp].index[-1]
            seqDf = pd.concat([dfs[i][endIndex-seqPartLen:endIndex], seqDf])
            lastTimestamp = seqDf["timestamp"].iloc[0]
        

        label = seqDf['target'].iloc[-1]
        seqDf = seqDf[["BTC_close", "BTC_volume", "BTC_HLPercent", "timescale"]]
        seq = seqDf.values.tolist()
        if(len(seq) == SEQ_LEN):
            sequences.append([seq, label])

        finestEndIndex -= CANDLES_SHIFT

    return sequences[::-1]
    



balance

In [3]:
def balance(seqs):
    
    buys = []
    sells = []
    holds = []
    for seq in seqs:
        if seq[1] == 0:
            sells.append(seq)
        elif seq[1] == 1:
            buys.append(seq)
        else:
          holds.append(seq)

    print("before balancing:")
    print("buys:", len(buys), ", sells:", len(sells), ", holds:", len(holds))

    smallest = min(len(buys), len(sells), len(holds))
    buys = buys[:smallest]
    sells = sells[:smallest]
    holds = holds[:smallest]

    seqsBalanced = buys+sells+holds
    return seqsBalanced

preprocessing

In [4]:
import numpy as np

means_300 = {"BTC_close": 0.000000690947, 
             "BTC_low": 0.0000006854884, 
             "BTC_high": 0.0000006769006, 
             "BTC_volume": 9.29817, 
             "BTC_HLPercent": 0.002693165}
stds_300 = {"BTC_close": 0.0003184924, 
             "BTC_low": 0.0002982297, 
             "BTC_high": 0.0002733320261, 
             "BTC_volume": 2.81534, 
             "BTC_HLPercent": 0.00318942}

means_900 = {"BTC_close": 0.0000020544577, 
             "BTC_low": 0.000002046661877, 
             "BTC_high": 0.00000201864122,  
             "BTC_volume": 10.884899, 
             "BTC_HLPercent": 0.005028761}
stds_900 = {"BTC_close": 0.000519522, 
             "BTC_low": 0.00050422044, 
             "BTC_high": 0.0004491347, 
             "BTC_volume": 2.2087313, 
             "BTC_HLPercent": 0.005339183}

means_3600 = {"BTC_close": 0.00000814210187, 
             "BTC_low": 0.0000081886629, 
             "BTC_high": 0.0000079834025, 
             "BTC_volume": 12.5666656, 
             "BTC_HLPercent": 0.01051739}
stds_3600 = {"BTC_close": 0.00098108448, 
             "BTC_low": 0.0010199776, 
             "BTC_high": 0.000852822218, 
             "BTC_volume": 1.83560254, 
             "BTC_HLPercent": 0.0102345275}

means_21600 = {"BTC_close": 0.0000483701454, 
             "BTC_low": 0.0000490174551, 
             "BTC_high": 0.00004777989, 
             "BTC_volume": 14.559, 
             "BTC_HLPercent": 0.02660019}
stds_21600 = {"BTC_close": 0.0022161646, 
             "BTC_low": 0.002430053655, 
             "BTC_high": 0.00197076029, 
             "BTC_volume": 1.6195640409, 
             "BTC_HLPercent": 0.022443346231}


def preprocess(df, means, stds):

    for col in ["BTC_close", "BTC_low", "BTC_high"]:
        df[col] = np.log(df[col])
        df[col] = df[col].pct_change()
        df.dropna(inplace=True)
        mean = means[col]
        std = stds[col]
        df[col] = (df[col] - mean) / std
    
    
    df["BTC_volume"] = df["BTC_volume"].replace(0, 1)
    df["BTC_volume"] = np.log(df["BTC_volume"])
    #df["BTC_volume"] = df["BTC_volume"].pct_change()   # taking the pct change somehow makes it worse
    #df.dropna(inplace=True)                            # taking the pct change somehow makes it worse
    mean = means["BTC_volume"]
    std = stds["BTC_volume"]
    df["BTC_volume"] = (df["BTC_volume"] - mean) / std


    mean = means["BTC_HLPercent"]
    std = stds["BTC_HLPercent"]
    df["BTC_HLPercent"] = (df["BTC_HLPercent"] - mean) / std


    return df


X y split

In [5]:
def split(seqWithTarget):

    X = []
    y = []
    for seq, target in seqWithTarget:
        X.append(np.asarray(seq).astype(np.float32))
        y.append(target)

    return np.array(X), np.array(y)

# DF manipulation, build training sets:

In [6]:
import pandas as pd

df_300_classified = pd.read_csv("data/aligned2/HistoricalDataClassified_2016_2023_300_ov40_th04.csv")
df_900 = pd.read_csv("data/aligned2/HistoricalData_2016_2023_900.csv")
df_3600 = pd.read_csv("data/aligned2/HistoricalData_2016_2023_3600.csv")
df_21600 = pd.read_csv("data/aligned2/HistoricalData_2016_2023_21600.csv")


df_300_classified["timescale"] = 5
df_900["timescale"] = 15
df_3600["timescale"] = 60
df_21600["timescale"] = 360

print(df_300_classified)

         timestamp  BTC_close   BTC_low  BTC_high  BTC_volume  BTC_average  \
0       1467331500     683.00    683.00    685.00      610.98       684.36   
1       1467331800     683.00    683.00    684.99     4207.54       683.04   
2       1467332100     683.80    682.94    684.00      832.24       683.84   
3       1467332400     682.93    682.93    683.80      731.29       683.74   
4       1467332700     682.93    682.93    682.93      339.97       682.93   
...            ...        ...       ...       ...         ...          ...   
710044  1680344400   28423.92  28416.67  28440.89     4032.00     28424.22   
710045  1680344700   28402.64  28401.90  28426.65     1226.36     28409.15   
710046  1680345000   28418.48  28403.11  28418.48     4559.18     28406.68   
710047  1680345300   28412.62  28407.49  28422.84      388.83     28413.01   
710048  1680345600   28422.07  28384.76  28422.09      306.02     28407.02   

        BTC_HLPercent  target  timescale  
0            0.00292

In [7]:
df_300_classified_pp = preprocess(df_300_classified, means_300, stds_300)
df_900_pp = preprocess(df_900, means_900, stds_900)
df_3600_pp = preprocess(df_3600, means_3600, stds_3600)
df_21600_pp = preprocess(df_21600, means_21600, stds_21600)
print(df_300_classified_pp)

         timestamp  BTC_close   BTC_low  BTC_high  BTC_volume  BTC_average  \
3       1467332400  -0.614533 -0.009822 -0.166373   -0.960225       683.74   
4       1467332700  -0.002169 -0.002299 -0.716016   -1.232289       682.93   
5       1467333000  -0.002169 -0.002299  1.694099   -0.655181       683.90   
6       1467333300   0.532917 -0.002299 -1.075060   -2.597081       683.29   
7       1467333600   0.911576  0.569143  1.062240   -1.698065       684.99   
...            ...        ...       ...       ...         ...          ...   
710044  1680344400  -0.180593  0.238298  0.002541   -0.353830     28424.22   
710045  1680344700  -0.231476 -0.172297 -0.181135   -0.776590     28409.15   
710046  1680345000   0.168546  0.011632 -0.105025   -0.310183     28406.68   
710047  1680345300  -0.065311  0.048123  0.052255   -1.184591     28413.01   
710048  1680345600   0.099650 -0.264044 -0.011891   -1.269658     28407.02   

        BTC_HLPercent  target  timescale  
3           -0.44549

In [8]:
#standardize timescale column
mean = 110
std = 145.8167
df_300_classified_pp["timescale"] = (df_300_classified_pp["timescale"] - mean) / std
df_900_pp["timescale"] = (df_900_pp["timescale"] - mean) / std
df_3600_pp["timescale"] = (df_3600_pp["timescale"] - mean) / std
df_21600_pp["timescale"] = (df_21600_pp["timescale"] - mean) / std
print(df_300_classified_pp)

         timestamp  BTC_close   BTC_low  BTC_high  BTC_volume  BTC_average  \
3       1467332400  -0.614533 -0.009822 -0.166373   -0.960225       683.74   
4       1467332700  -0.002169 -0.002299 -0.716016   -1.232289       682.93   
5       1467333000  -0.002169 -0.002299  1.694099   -0.655181       683.90   
6       1467333300   0.532917 -0.002299 -1.075060   -2.597081       683.29   
7       1467333600   0.911576  0.569143  1.062240   -1.698065       684.99   
...            ...        ...       ...       ...         ...          ...   
710044  1680344400  -0.180593  0.238298  0.002541   -0.353830     28424.22   
710045  1680344700  -0.231476 -0.172297 -0.181135   -0.776590     28409.15   
710046  1680345000   0.168546  0.011632 -0.105025   -0.310183     28406.68   
710047  1680345300  -0.065311  0.048123  0.052255   -1.184591     28413.01   
710048  1680345600   0.099650 -0.264044 -0.011891   -1.269658     28407.02   

        BTC_HLPercent  target  timescale  
3           -0.44549

split into dfs with SEQ_LEN rows

In [9]:
sequences = composeTimeDecaySequences([df_300_classified_pp, df_900_pp, df_3600_pp, df_21600_pp], [300, 900, 3600, 21600])

100%|██████████| 141745/141745 [52:50<00:00, 44.71it/s]  


In [10]:
print("num sequences:", len(sequences))

num sequences: 141737


seperate training and validation

In [11]:
seqsTraining = sequences[:(int(len(sequences) * (1-VALIDATION_PCT)))].copy()
seqsValidation = sequences[(int(len(sequences) * (1-VALIDATION_PCT))):].copy()

shuffle

In [12]:
import random

random.shuffle(seqsTraining)
random.shuffle(seqsValidation)

balance buys/sells/holds

In [13]:
seqsTrainingBalanced = balance(seqsTraining)
seqsValidationBalanced = balance(seqsValidation)

before balancing:
buys: 13947 , sells: 13197 , holds: 86245
before balancing:
buys: 2626 , sells: 2655 , holds: 23067


shuffle

In [14]:
random.shuffle(seqsTrainingBalanced)
random.shuffle(seqsValidationBalanced)

In [15]:
#print([type(seq) for seq, target in seqsValidationBalanced])
#print([np.shape(seq) for seq, target in seqsValidationBalanced if isinstance(seq, np.ndarray) or isinstance(seq, list)])


split sequence from label

In [16]:
train_x, train_y = split(seqsTrainingBalanced)
validation_x, validation_y = split(seqsValidationBalanced)



In [17]:
print(type(train_x))
print(train_x.shape)
print(type(validation_x))
print(validation_x.shape)

<class 'numpy.ndarray'>
(39591, 60, 4)
<class 'numpy.ndarray'>
(7878, 60, 4)


In [19]:
import pickle

with open(f'data/pkl/{NAME}/{NAME}_train_x.pkl', 'wb') as file:
    pickle.dump(train_x, file)
with open(f'data/pkl/{NAME}/{NAME}_train_y.pkl', 'wb') as file:
    pickle.dump(train_y, file)
with open(f'data/pkl/{NAME}/{NAME}_validation_x.pkl', 'wb') as file:
    pickle.dump(validation_x, file)
with open(f'data/pkl/{NAME}/{NAME}_validation_y.pkl', 'wb') as file:
    pickle.dump(validation_y, file)

In [ ]:
#import pickle
#
#with open(f"data/pkl/{NAME}/{NAME}_train_x.pkl", "rb") as file:
#    train_x = pickle.load(file)
#with open(f"data/pkl/{NAME}/{NAME}_train_y.pkl", "rb") as file:
#    train_y = pickle.load(file)
#with open(f"data/pkl/{NAME}/{NAME}_validation_x.pkl", "rb") as file:
#    validation_x = pickle.load(file)
#with open(f"data/pkl/{NAME}/{NAME}_validation_y.pkl", "rb") as file:
#    validation_y = pickle.load(file)

# Model:

In [20]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


hyper parameters bounds

In [21]:
EPOCHS = 100

batchSize = 96
layers = 2
nodes = 256
denseNodes = 128
dropOut = 0.88
learningRate = 0.0001
decay = 1e-06


In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import os
from tqdm.keras import TqdmCallback
import datetime

# model
model = Sequential()

for _ in range(layers-1):
  model.add(LSTM(nodes, activation="tanh", recurrent_activation = 'sigmoid', recurrent_dropout = 0, unroll = False, use_bias = True, input_shape=(train_x.shape[1:]), return_sequences=True))
  model.add(Dropout(dropOut))
  #model.add(BatchNormalization())

model.add(LSTM(nodes, activation="tanh", recurrent_activation = 'sigmoid', recurrent_dropout = 0, unroll = False, use_bias = True, input_shape=(train_x.shape[1:])))
model.add(Dropout(dropOut))
#model.add(BatchNormalization())

model.add(Dense(denseNodes, activation="relu"))
model.add(Dropout(dropOut))

model.add(Dense(3, activation="softmax"))

# opt  
opt = tf.keras.optimizers.Adam(lr=learningRate) # decay?

model.compile(loss="sparse_categorical_crossentropy",
              optimizer = opt,
              metrics=["accuracy"])

filename = NAME + "-{epoch:02d}"
filepath = f"models/{filename}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1) # saves every epoch

# Prepare TensorBoard callback
log_dir = f"logs/fit/{NAME}" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# train
history = model.fit(
  train_x, train_y,
  batch_size = batchSize,
  epochs = EPOCHS,
  validation_data=(validation_x, validation_y),
  callbacks=[checkpoint, TqdmCallback(verbose=0), tensorboard_callback])


c:\Users\Johannes\miniconda3\envs\Quant\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


c:\Users\Johannes\miniconda3\envs\Quant\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
  0%|          | 0/100 [00:00<?, ?epoch/s]

Epoch 1/100
413/413 [==============================] - ETA: 0s - loss: 1.1069 - accuracy: 0.3821

  1%|          | 1/100 [00:13<21:56, 13.29s/epoch, loss=1.11, accuracy=0.382, val_loss=1.05, val_accuracy=0.51]

413/413 [==============================] - 13s 27ms/step - loss: 1.1069 - accuracy: 0.3821 - val_loss: 1.0527 - val_accuracy: 0.5099
Epoch 2/100
411/413 [============================>.] - ETA: 0s - loss: 1.0466 - accuracy: 0.4586

...

 99%|█████████▉| 99/100 [17:26<00:10, 10.46s/epoch, loss=0.861, accuracy=0.627, val_loss=0.908, val_accuracy=0.596]

413/413 [==============================] - 10s 25ms/step - loss: 0.8606 - accuracy: 0.6266 - val_loss: 0.9085 - val_accuracy: 0.5960
Epoch 100/100
413/413 [==============================] - ETA: 0s - loss: 0.8596 - accuracy: 0.6267

100%|██████████| 100/100 [17:36<00:00, 10.47s/epoch, loss=0.86, accuracy=0.627, val_loss=0.913, val_accuracy=0.603]

413/413 [==============================] - 11s 25ms/step - loss: 0.8596 - accuracy: 0.6267 - val_loss: 0.9131 - val_accuracy: 0.6028


100%|██████████| 100/100 [17:36<00:00, 10.57s/epoch, loss=0.86, accuracy=0.627, val_loss=0.913, val_accuracy=0.603]
